- https://nijianmo.github.io/amazon/index.html
- https://jmcauley.ucsd.edu/data/amazon/ 
- https://colab.research.google.com/drive/1Zv6MARGQcrBbLHyjPVVMZVnRWsRnVMpV

In [1]:
prfx = 'prep_20210307A1'

# get data

    download from http://localhost:8080/notebooks/git/product-category/notebooks/secrets_get_amazon_data.ipynb

In [2]:
import pandas as pd
import gzip
import json

In [3]:
pdata = "/data/git/product-category/data/amazon_meta_data"
!ls -hl {pdata}

total 13G
-rw-rw-r-- 1 ubuntu ubuntu   30M Dec  2 00:07 meta_AMAZON_FASHION.json.gz
-rw-rw-r-- 1 ubuntu ubuntu  9.6M Aug  7  2020 meta_All_Beauty.json.gz
-rw-rw-r-- 1 ubuntu ubuntu   57M Aug  7  2020 meta_Appliances.json.gz
-rw-rw-r-- 1 ubuntu ubuntu   57M Aug  7  2020 meta_Appliances.json.gz.1
-rw-rw-r-- 1 ubuntu ubuntu  125M Aug  7  2020 meta_Arts_Crafts_and_Sewing.json.gz
-rw-rw-r-- 1 ubuntu ubuntu  1.5G Aug  7  2020 meta_Automotive.json.gz
-rw-rw-r-- 1 ubuntu ubuntu  1.2G Aug  7  2020 meta_Books.json.gz
-rw-rw-r-- 1 ubuntu ubuntu  1.2G Aug  7  2020 meta_Books.json.gz.1
-rw-rw-r-- 1 ubuntu ubuntu  153M Aug  7  2020 meta_CDs_and_Vinyl.json.gz
-rw-rw-r-- 1 ubuntu ubuntu  333M Aug  7  2020 meta_Cell_Phones_and_Accessories.json.gz
-rw-rw-r-- 1 ubuntu ubuntu  1.5G Dec  2 00:18 meta_Clothing_Shoes_and_Jewelry.json.gz
-rw-rw-r-- 1 ubuntu ubuntu   12M Aug  7  2020 meta_Digital_Music.json.gz
-rw-rw-r-- 1 ubuntu ubuntu  1.2G Aug  7  2020 meta_Electronics.json.gz
-rw-rw-r-- 1 ubuntu ubuntu  37

In [4]:
fnms = !ls {pdata}
fnms = [o for o in fnms if o.endswith('.json.gz')]
fnms

['meta_AMAZON_FASHION.json.gz',
 'meta_All_Beauty.json.gz',
 'meta_Appliances.json.gz',
 'meta_Arts_Crafts_and_Sewing.json.gz',
 'meta_Automotive.json.gz',
 'meta_Books.json.gz',
 'meta_CDs_and_Vinyl.json.gz',
 'meta_Cell_Phones_and_Accessories.json.gz',
 'meta_Clothing_Shoes_and_Jewelry.json.gz',
 'meta_Digital_Music.json.gz',
 'meta_Electronics.json.gz',
 'meta_Gift_Cards.json.gz',
 'meta_Grocery_and_Gourmet_Food.json.gz',
 'meta_Home_and_Kitchen.json.gz',
 'meta_Industrial_and_Scientific.json.gz',
 'meta_Kindle_Store.json.gz',
 'meta_Luxury_Beauty.json.gz',
 'meta_Magazine_Subscriptions.json.gz',
 'meta_Movies_and_TV.json.gz',
 'meta_Musical_Instruments.json.gz',
 'meta_Office_Products.json.gz',
 'meta_Patio_Lawn_and_Garden.json.gz',
 'meta_Pet_Supplies.json.gz',
 'meta_Prime_Pantry.json.gz',
 'meta_Software.json.gz',
 'meta_Sports_and_Outdoors.json.gz',
 'meta_Tools_and_Home_Improvement.json.gz',
 'meta_Toys_and_Games.json.gz',
 'meta_Video_Games.json.gz']

In [5]:
domains = [o.strip('meta_').strip('.json.gz') for o in fnms]
dmn2fnm = dict(zip(domains, fnms))
dmn2fnm

{'AMAZON_FASHION': 'meta_AMAZON_FASHION.json.gz',
 'All_Beauty': 'meta_All_Beauty.json.gz',
 'Appliance': 'meta_Appliances.json.gz',
 'Arts_Crafts_and_Sewi': 'meta_Arts_Crafts_and_Sewing.json.gz',
 'Automotive': 'meta_Automotive.json.gz',
 'Book': 'meta_Books.json.gz',
 'CDs_and_Vinyl': 'meta_CDs_and_Vinyl.json.gz',
 'Cell_Phones_and_Accessorie': 'meta_Cell_Phones_and_Accessories.json.gz',
 'Clothing_Shoes_and_Jewelry': 'meta_Clothing_Shoes_and_Jewelry.json.gz',
 'Digital_Music': 'meta_Digital_Music.json.gz',
 'Electronic': 'meta_Electronics.json.gz',
 'Gift_Card': 'meta_Gift_Cards.json.gz',
 'Grocery_and_Gourmet_Food': 'meta_Grocery_and_Gourmet_Food.json.gz',
 'Home_and_Kitche': 'meta_Home_and_Kitchen.json.gz',
 'Industrial_and_Scientific': 'meta_Industrial_and_Scientific.json.gz',
 'Kindle_Store': 'meta_Kindle_Store.json.gz',
 'Luxury_Beauty': 'meta_Luxury_Beauty.json.gz',
 'Magazine_Subscripti': 'meta_Magazine_Subscriptions.json.gz',
 'Movies_and_TV': 'meta_Movies_and_TV.json.gz',
 

In [6]:
KEYS2USE = set(['category', 'description', 'title', 'brand', 'feature', 'asin'])

In [7]:
# https://colab.research.google.com/drive/1Zv6MARGQcrBbLHyjPVVMZVnRWsRnVMpV#scrollTo=7igYuRaV4bF7
    
### load the meta data
def get_meta_data(domain, nrows=None):
    keys2use = KEYS2USE
    fnm = dmn2fnm[domain]
    data = []
    with gzip.open(f'{pdata}/{fnm}') as f:
        for i,l in enumerate(f):
            dat = json.loads(l.strip())
            dat = {k:v for k,v in dat.items() if k in keys2use}
            data.append(dat)
            if nrows and i>nrows: break

    # total length of list, this number equals total number of products
#     print("len(data)", len(data))

    # convert list into pandas dataframe
    df = pd.DataFrame.from_dict(data)
#     print("len(df)", len(df))

    ### remove rows with unformatted title (i.e. some 'title' may still contain html style content)
    df3 = df.fillna('')
    df4 = df3[df3.title.str.contains('getTime')] # unformatted rows
    df5 = df3[~df3.title.str.contains('getTime')] # filter those unformatted rows
#     print("len(df4)", len(df4))
#     print("len(df5)", len(df5))

    return df5

In [8]:
df = get_meta_data(domains[3], nrows=10000)
df.head(3)

,category,description,title,brand,feature,asin
0,"[Arts, Crafts & Sewing, Sewing, Trim & Embelli...",[The patch features the sweaty masculine hands...,You Son of a Bitch! 1987 Embroidered Patch,Honchosfx,[You son of a bitch patch - exclusive to Honch...,6665560953
1,"[Arts, Crafts & Sewing, Crafting, Paper & Pape...","[With 5 packs stars folding paper, each pack a...",Origami Stars Papers Package 1H (5 packs),,[],7000000376
2,"[Arts, Crafts & Sewing, Painting, Drawing & Ar...",[],Yi De Ge Chinese Calligraphy Sumi Drawing Blac...,MasterChinese,[],7000001089


# preprocess data

## concate

In [9]:
dfs = []
for dmn in domains:
    print(dmn)
    df = get_meta_data(dmn, nrows=None)
    if set(df.columns)!= KEYS2USE: continue
    df = df[df.category.apply(len)>0]
    if len(df)==0: continue
    print(df.shape)
    df['domain'] = dmn
    display(df.sample(2))
    dfs.append(df)

AMAZON_FASHION
All_Beauty
Appliance
(29639, 6)


,category,description,title,brand,feature,asin,domain
19305,"[Appliances, Parts & Accessories, Refrigerator...",[This is a genuine replacement part. The model...,Frigidaire 218947201 Fill Tube Refrigerator,Frigidaire,"[Manufacturer Model #218947201, Genuine Replac...",B00M25GXRU,Appliance
3771,"[Appliances, Parts & Accessories, Refrigerator...",[Whirlpool Whirlpool Refrigerator Icemaker Ice...,Maytag Refrigerator Icemaker Ice Maker &amp; W...,Maytag,[Whirlpool- Genuine Replacement Part- Certifie...,B001TIWJQ0,Appliance


Arts_Crafts_and_Sewi
(284889, 6)


,category,description,title,brand,feature,asin,domain
69137,"[Arts, Crafts & Sewing, Sewing, Trim & Embelli...",[C&D VISIONARY INC-C&D Visionary Iron-On Patch...,Application Giraffe Patch,Application,"[Iron-On or Sew to Any Garment, Top Quality, D...",B005P1XBGE,Arts_Crafts_and_Sewi
252968,"[Arts, Crafts & Sewing, Sewing, Sewing Notions...","[Sizes: 60mm(2 3/8"") X 60mm(2 3/8"")<br> Quanti...",Bluemoona 20 PCS - Empty Plastic Wire Spools B...,Sewing,"[Sizes: 60mm(2 3/8"") X 60mm(2 3/8""), Color: BL...",B014ZO9JXU,Arts_Crafts_and_Sewi


Automotive
(897587, 6)


...

,category,description,title,brand,feature,asin,domain
8141,"[Tools & Home Improvement, Electrical, Switche...",[Control holiday lights automatically by plugg...,Ingraham Outdoor Countdown Timer,Salton,"[8 timed settings, All weather case, Heavy dut...",B00013K2Q8,Tools_and_Home_Improvement
5206,"[Tools & Home Improvement, Power & Hand Tools,...",[Double tooth for longer life and improved dur...,DEWALT D180044 2-3/4-Inch Hole Saw,DEWALT,"[The product is 2-3/4"" 70MM Bi Hole Saw, Easy ...",B00005LEZI,Tools_and_Home_Improvement


Toys_and_Game
(571640, 6)


,category,description,title,brand,feature,asin,domain
187903,"[Toys & Games, Sports & Outdoor Play, Bubbles]","[Bubble Fish Look, listen, feel and pour! Ple...",Ambi Bubble Fish,Ambi Toys,"[The essential toy for bath time fun., This co...",B004M8ROI2,Toys_and_Game
354823,"[Toys & Games, Baby & Toddler Toys, Music & So...",[Play & Discover with this educational toy Col...,PlayGo First Discoveries Toy,PlayGo,[Watch your baby enjoy and discover a variety ...,B00EVY1NRA,Toys_and_Game


Video_Game
(82394, 6)


,category,description,title,brand,feature,asin,domain
52605,"[Video Games, Wii, Accessories, Cables & Adapt...",[Extends the effective range of your console t...,2x 6FT Controller Extension Cable For Wii Game...,by\n \n KMD,[Extends the effective range of your console t...,B005SN8HFO,Video_Game
48962,"[Video Games, PC, Games, </span></span></span>]","[<div>, Princess Jessa, along with the loyal k...",Shining in the Darkness [Download],by\n \n Sega,[],B004J4WZI8,Video_Game


In [10]:
df = pd.concat(dfs)
df.shape

(13706665, 7)

In [11]:
df.sample(3)

,category,description,title,brand,feature,asin,domain
1526888,"[Clothing, Shoes & Jewelry, Luggage & Travel G...",[300d polyester duffle color blocked with soli...,Billabong Men's Sierra Grands Duffle,,"[100% Polyester, Imported, Zipper closure, 30""...",B00N9RD7FQ,Clothing_Shoes_and_Jewelry
662035,"[Automotive, Replacement Parts, Transmission &...",[],Genuine Honda 46971-SR3-A00 Clutch Master Cyli...,Honda,"[Del Sol '93-97, Genuine OEM, Direct fit, Genu...",B00LIUKAH4,Automotive
100280,"[Musical Instruments, Microphones & Accessorie...","[<br>Special design for chatting over QQ, MSN,...",Vktech&reg; Super USB 2.0 Condenser Microphone...,Vktech,"[Special design for chatting over QQ, MSN, SKY...",B00UV7CZP0,Musical_Instrument


## cleanup

### category

In [12]:
%%time
def try2eval(x):
    try:
        x = eval(x)
        x = (o.replace('|','') for o in x)
        return '|'.join(x)
    except SyntaxError:
        return ""

df.category = df.category.astype(str)
df.category = df.category.apply(try2eval)
print("df.shape:", df.shape)

df.shape: (13706665, 7)
CPU times: user 3min 36s, sys: 4.95 s, total: 3min 41s
Wall time: 3min 40s


### text

In [13]:
import re

def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

In [14]:
%%time
df.fillna('', inplace=True)

for col in ['description', 'feature']:
    df[col] = df[col].apply(lambda x: "\n".join(x) if len(x) else "")

for col in ['description', 'title', 'brand', 'feature',]:
    df[col] = df[col].apply(cleanhtml)

CPU times: user 3min 20s, sys: 4.94 s, total: 3min 25s
Wall time: 3min 25s


### truncate

In [15]:
import numpy as np
for col in ['description', 'title', 'brand', 'feature',]:
    leng = df[col].apply(len)
    print(col, np.percentile(leng, 95)) 

description 1666.0
title 145.0
brand 34.0
feature 754.0


In [16]:
def trunc(x): return x[:1000]    

In [17]:
%%time
df['txt'] = df.title.apply(trunc) + " " + df.brand.apply(trunc) + " " + df.description.apply(trunc) + " " + df.feature.apply(trunc)




CPU times: user 50.9 s, sys: 11.9 s, total: 1min 2s
Wall time: 1min 10s


In [18]:
leng = df.txt.apply(len)
print(leng.max(), np.percentile(leng, 95)) 

3113 1536.0


# train val split

In [19]:
import numpy as np

np.random.seed(101)
df['is_validation'] = np.random.choice(2, size=len(df), p=[0.85,0.15])

In [20]:
df.is_validation.value_counts()

0    11649852
1     2056813
Name: is_validation, dtype: int64

In [21]:
df.sample(3)

,category,description,title,brand,feature,asin,domain,txt,is_validation
173884,CDs & Vinyl|Alternative Rock|New Wave & Post-P...,VARIOUS ARTISTS - 1980S: MODERN ROCK OF THE 80...,Hardest Hits: Modern Rock of the 80's,Various Artists,,B00009PXXA,CDs_and_Vinyl,Hardest Hits: Modern Rock of the 80's Various ...,0
1251629,"Clothing, Shoes & Jewelry|Women|Shoes|Boots|Kn...",The Aquatalia by Marvin K. collection is manuf...,Aquatalia Women's Helena Dress Boot,Aquatalia,100% Leather\nImported\nSynthetic sole\nShaft ...,B00ISSXHZG,Clothing_Shoes_and_Jewelry,Aquatalia Women's Helena Dress Boot Aquatalia ...,0
934399,"Clothing, Shoes & Jewelry|Women|Contemporary &...",Enjoy all-star footwear from one of the most r...,kate spade new york Women's Cris Pump,,100% Leather\nImported\nLeather sole\nHeel mea...,B00DSLK594,Clothing_Shoes_and_Jewelry,kate spade new york Women's Cris Pump Enjoy a...,0


# save

In [22]:
%%time
df.to_csv(f'../data/data__{prfx}.csv', index=False)

CPU times: user 9min 42s, sys: 38.6 s, total: 10min 20s
Wall time: 10min 21s


In [23]:
ls -hl ../data/data__{prfx}.csv

-rw-rw-r-- 1 ubuntu ubuntu 20G Mar  7 15:35 ../data/data__prep_20210307A1.csv


## make a sample

In [24]:
import numpy as np
np.random.seed(101)
df_sample = df.sample(10000)

In [25]:
df_sample.to_csv(f'../data/data_sample__{prfx}.csv', index=False)

## make more samples

In [51]:
%%time
df = pd.read_csv(f'../data/data__{prfx}.csv', error_bad_lines=False)

b'Skipping line 2057200: expected 9 fields, saw 22\n'


KeyboardInterrupt: 

In [52]:
df.shape

(1000000, 9)

In [43]:
df.sample().txt.values

array(['Carl Pencil Sharpener CC-2000 Red Carl Pencil Brand Karujimu-ki \nProduct Number A5PR-R \nLong-life products that continue to use a long time color red. Cutting instrument pencil functional timeless durable. \nClassic body of sheet metal working, I was stuck in the texture and color. \nBody sheet metal products. Rubber chuck use without the scratches in pencil. \nBlade sharpener made in Japan. \nCommodity Description \nThe durable, and fully renewable, it is beautiful pencil sharpener using material of ultimate returns to the soil all the "iron" in the end. In addition, the part that sandwich to avoid scratching the surface of the pencil sharpening, use a chuck of rubber to hold firmly soft. It was made up of special steel, along with the blade, keeping long-term high functionality sharpener is superior in accuracy and durability. . .  I am using another product color in some detail image. . Size: thickness = 132mm Vertical Horizontal = 75mm = 125mm. Item color trend: red. . 12

In [28]:
np.random.seed(101)
for sz in (int(o) for o in (1e4, 1e5, 5e5, 1e6, 5e6)):
    df_sample = df.sample(sz)
    df_sample.to_csv(f'../data/data_sample_{sz}__{prfx}.csv', index=False)

KeyboardInterrupt: 